<a href="https://colab.research.google.com/github/vochicong/ai-memo/blob/master/SentencePiece_JA_wagahaiwa_nekodearu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 吾輩は猫である SentencePiece 

自然言語処理において、文章を形態素解析したり、トークン化したりする作業が欠かせない。
ここでは夏目漱石の「吾輩は猫である」を[SentencePiece](https://github.com/google/sentencepiece/blob/6065c4a24f89401899595e14af826632d876587c/src/unigram_model_trainer_test.cc)
でトークン化してみましょう。

## インストール

In [1]:
!pip install sentencepiece

## データダウンロード

「吾輩は猫である」のテキストデータをダウンロード

In [2]:
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/wagahaiwa_nekodearu.txt
!wc -lm wagahaiwa_nekodearu.txt

--2019-11-08 05:47:50--  https://raw.githubusercontent.com/google/sentencepiece/master/data/wagahaiwa_nekodearu.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1120576 (1.1M) [text/plain]
Saving to: ‘wagahaiwa_nekodearu.txt.6’

wagahaiwa_nekodearu 100%[===================>]   1.07M  --.-KB/s    in 0.09s   

2019-11-08 05:47:50 (12.3 MB/s) - ‘wagahaiwa_nekodearu.txt.6’ saved [1120576/1120576]

   2344  377212 wagahaiwa_nekodearu.txt


In [3]:
!head wagahaiwa_nekodearu.txt
!echo ......
!tail wagahaiwa_nekodearu.txt

吾輩は猫である
夏目漱石
-------------------------------------------------------
【テキスト中に現れる記号について】
《》：ルビ
（例）吾輩《わがはい》は猫である
｜：ルビの付く文字列の始まりを特定する記号
（例）一番｜獰悪《どうあく》な種族であった
［＃］：入力者注　主に外字の説明や、傍点の位置の指定
　　　（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）
......
底本の親本：「筑摩全集類聚版夏目漱石全集」筑摩書房
　　　1971（昭和46）年4月〜1972（昭和47）年1月
初出：「ホトトギス」
　　　1905（明治38）年1月〜8月
入力：柴田卓治
校正：渡部峰子（一）、おのしげひこ（二、五）、田尻幹二（三）、高橋真也（四、七、八、十、十一）、しず（六）、瀬戸さえ子（九）
1999年9月17日公開
2015年2月3日修正
青空文庫作成ファイル：
このファイルは、インターネットの図書館、青空文庫（http://www.aozora.gr.jp/）で作られました。入力、校正、制作にあたったのは、ボランティアの皆さんです。


## SentencePiece モデルの学習

トークン化のやり方を覚えさせるために、「吾輩は猫である」のテキストを与えて、
SentencePieceに学習させます。

In [4]:
import sentencepiece as spm

# Train sentencepiece model. Params taken from https://github.com/google/sentencepiece/blob/d4dd947fe71c4fa4ee24ad8297beee32887d8828/src/unigram_model_trainer_test.cc
spm.SentencePieceTrainer.train('--input=wagahaiwa_nekodearu.txt --model_prefix=wagahaiwa_nekodearu --vocab_size=8000 '
  '--normalization_rule_name=identity --model_type=unigram  --max_sentence_length=2048')

# makes segmenter instance and loads the model file
sp = spm.SentencePieceProcessor()
sp.load('wagahaiwa_nekodearu.model')

True

## トークン化の実施

下に定義する text をトークン化すると、 pieces に分解されることを確認しましょう。
また、IDも取得できます。

その後、piecesまたはIDから元の文章を正しく復元 (decode) できることを確認します。

In [0]:
text = (
    "吾輩《わがはい》は猫である。"
    "名前はまだ無い。"
    "どこで生れたかとんと見当《けんとう》がつかぬ。"
    )
pieces = [
          '▁', '吾輩', '《', 'わが', 'はい', '》', 'は', '猫', 'である', '。',  
          '名前', 'はまだ', '無い', '。', 
          'どこ', 'で', '生', 'れた', 'か', 'とん', 'と', 
          '見当', '《', 'けん', 'とう', '》', 'が', 'つか', 'ぬ', '。']
ids = [135, 449, 4, 2388, 143, 3, 16, 184, 35, 5, 679, 1950, 2578, 5, 281, 18, 
       157, 1020, 17, 424, 14, 2401, 4, 302, 236, 3, 13, 277, 152, 5]       

In [0]:
# encode
assert sp.encode_as_pieces(text) == pieces
assert sp.encode_as_ids(text) == ids

In [0]:
# decode
assert sp.decode_pieces(pieces) == sp.decode_ids(ids) == text

## まとめ

SentencePieceで簡単に「吾輩は猫である」のテキストをトークンに分解できました。
また、バラバラに分解されたピースを使って、元の文章を再構築できました。